In [1]:
import os
import sys
import librosa
import tflearn
import wave
import tensorflow as tf
import librosa.display
import IPython.display
import numpy as np
import speech_data
from pydub import AudioSegment as audio

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


hdf5 not supported (please install/reinstall h5py)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# now put all of the mfccs into an array
data = '/home/cc/working/data/devclean_seg/'
os.chdir(data)
speakers = speech_data.get_speakers(os.getcwd())
audio_files = os.listdir(os.getcwd())
mfccs = []
Y = []
for f in audio_files:
  Y.append(speech_data.one_hot_from_item(speech_data.speaker(f), speakers))
  y, sr = librosa.load(f)
  mfccs.append(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))

number of files: 1797
(10, ' speakers: ', ['Speaker1', 'Speaker0', 'Speaker3', 'Speaker2', 'Speaker5', 'Speaker4', 'Speaker7', 'Speaker6', 'Speaker9', 'Speaker8'])


In [3]:
net = tflearn.input_data(shape=[None, 13, 44])
net = tflearn.fully_connected(net, 64)
net = tflearn.dropout(net, 0.6)
net = tflearn.fully_connected(net, len(speakers), activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

In [4]:
model = tflearn.DNN(net)
model.fit(mfccs, Y, n_epoch=1000, show_metric=True, snapshot_step=100)

Training Step: 29000  | total loss: 0.77237
| Adam | epoch: 1000 | loss: 0.77237 - acc: 0.9640 -- iter: 1797/1797
Training Step: 29000  | total loss: 0.77237
| Adam | epoch: 1000 | loss: 0.77237 - acc: 0.9640 -- iter: 1797/1797
--


In [5]:
os.chdir('/home/cc/working/models/devclean/')
model.save('devclean_train.tflearn')

In [6]:
os.chdir('/home/cc/working/data/devclean_test/')

test = []
for f1 in os.listdir(os.getcwd()):
  y, sr = librosa.load(f1)
  test.append(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
result=model.predict(test)
c = 0
for f,r in zip(os.listdir(os.getcwd()), result):
  res = speech_data.one_hot_to_item(r, speakers)
  if res in f:
    c = c + 1
acc = float(c) / float(len(test))
print('Test set accuracy: %s' %str(acc))

Test set accuracy: 0.89
